In [ ]:
import pandas as pd
import json

In [ ]:
df_redefine = pd.read_csv('./ariel_project_data_1Kdays.csv')
df_tagged = pd.read_json('./etherscan-tagged-data.json')
print(df_redefine.head())
df_tagged.head()

In [ ]:
df_redefine['other_contracts'] = df_redefine['other_contracts'].apply(lambda x: x.replace("'",'"'))
df_redefine['other_contracts'] = df_redefine['other_contracts'].apply(lambda x: json.loads(x))


In [ ]:
def parse_contracts(obj):
    x=[]
    for address in obj:
        y = address['smartContract']['address']['address']
        x.append(y)
    return x

df_redefine['other_created_address'] = df_redefine['other_contracts'].apply(lambda x: parse_contracts(x))

In [ ]:
df_redefine.columns

In [ ]:
df_redefine_expanded = df_redefine.loc[:,['token_creator','other_created_address','project_name']]
df_redefine_expanded

In [ ]:
df_redefine_expanded = df_redefine_expanded.explode('other_created_address')

In [ ]:
df_redefine_expanded

In [ ]:
df_etherscan = df_tagged.transpose()
df_etherscan.reset_index(inplace=True)
df_etherscan


In [ ]:
df_merge = pd.merge(df_redefine_expanded, df_etherscan, left_on="other_created_address",right_on="index", how="left")
df_is_tagged = df_merge.loc[df_merge['index'].notna()]

In [ ]:
len(df_redefine_expanded.drop_duplicates('other_created_address'))
len(df_etherscan.drop_duplicates('index'))

In [ ]:
df_is_tagged.set_index('index', inplace=True)


In [ ]:
df_is_tagged.drop('other_created_address', axis=1,inplace=True)

Data stats:

In [ ]:
print("Tokens analyzed: %s"%(len(df_redefine)))
print("Num of contracts created by token creator: %s"%(len(df_redefine_expanded)))
print("Tagged contracts analyzed: %s"%(len(df_etherscan)))
print("Num of created contracts that are tagged: %s"%(len(df_merge.loc[df_merge['index'].notna()])))

As can be seen - there is very few common contracts between the Redefine DF and Etherscan tagged data. 
To overcome this we will filter out contracts that haven't been used in the past year.

Additional ways for us to increase validation of contracts:
1. Scrape Etherscan up to date V
2. Add a date threshold 
3. Call contract name + call name function

In [ ]:
df_is_tagged.to_csv('first_results.csv')

We will verify from the tagged Etherscan names which are correct and which are incorrect

In [ ]:
filter = df_is_tagged.drop_duplicates('project_name')

In [109]:
remove_words = ["token",
"proxy",
"contract",
"work",
"protocol",
"governance",
"dao",
"finance"]



In [ ]:
def verify_name(p_name, labels,name):
    project_name = p_name.split()
    print(project_name)
    for name in project_name:
        if name in remove_words:
            project_name.remove(name)
    print(project_name)




df_is_tagged['verified'] = df_is_tagged['project_name','labels','name'].apply(lambda x: verify_name(x))

In [113]:
def verify_name(p_name, labels,name):
    project_name = p_name.split()
    lower_project_name = []
    for name in project_name:
        name = name.lower()
        name = name.
        lower_project_name.append(name)
        if name in remove_words:
            lower_project_name.remove(name)
    print(lower_project_name)

verify_name('Spell Token (SPELL)','','')

['spell', '(spell)']
